In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

sns.set_theme(style="darkgrid")
plt.rcParams['figure.figsize'] = [8, 4.5]
plt.rcParams['figure.dpi'] = 300
warnings.simplefilter(action='ignore', category=FutureWarning)

# CPMA

`Capital Asset Pricing Model (CAPM)` is a model that describes the relationship between the expected return and risk of investing in a security. \
It shows that the expected return on a security is equal to the risk-free return plus a risk premium, which is based on the beta of that security. \
Below is an illustration of the CAPM concept. (CFI TEAM)

In [3]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm

1- Specify the risky asset and the time horizon

In [4]:
RISKY_ASSET = 'AMZN'
MARKET_BENCHMARK = '^GSPC'
START_DATE = '2014-01-01'
END_DATE = '2018-12-31'

2- Download data from Yahoo Finance

In [6]:
df = yf.download([RISKY_ASSET, MARKET_BENCHMARK], start=START_DATE, end=END_DATE)
print(f'Download {df.shape[0]} rows of date')

[*********************100%%**********************]  2 of 2 completed
Download 1257 rows of date


3- Resample to monthly data and calculate simple returns

In [9]:
X = df['Adj Close'].rename(columns={RISKY_ASSET: 'asset', MARKET_BENCHMARK: 'market'}) \
                    .resample('M') \
                    .last() \
                    .pct_change() \
                    .dropna()

X.head()

,asset,market
Date,,
2014-02-28,0.009507,0.043117
2014-03-31,-0.071058,0.006932
2014-04-30,-0.095847,0.006201
2014-05-31,0.027685,0.021030
2014-06-30,0.039130,0.019058


4- Calculate beta using the covariance approach

In [11]:
covariance = X.cov().iloc[0,1]
benchmark_variance = X.market.var()
beta = covariance / benchmark_variance
beta

1.6296082618965537

5- Prepare the input and estimate CAPM as a linear regression

In [12]:
#separate target
y = X.pop('asset')

#add constant
X = sm.add_constant(X)

#define and fit the regression model
capm_model = sm.OLS(y, X).fit()

#print results
print(capm_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  asset   R-squared:                       0.365
Model:                            OLS   Adj. R-squared:                  0.354
Method:                 Least Squares   F-statistic:                     32.77
Date:                Wed, 14 Feb 2024   Prob (F-statistic):           4.05e-07
Time:                        17:09:53   Log-Likelihood:                 75.313
No. Observations:                  59   AIC:                            -146.6
Df Residuals:                      57   BIC:                            -142.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0178      0.009      1.950      0.0